In [ ]:
%pip install --upgrade --quiet langchain-core langchain-community langchain-openai langchain-groq langchain-ai21 docarray tiktoken


In [6]:
import os

# os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'
os.environ['GROQ_API_KEY'] = 'YOUR_GROQ_API'
os.environ["AI21_API_KEY"] = 'YOUR_AI21_API'

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_groq import ChatGroq
from langchain_ai21 import AI21Embeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["Marcos trabalhou na TOTVS", "ursos gostam de comer mel"],
    embedding=AI21Embeddings(),
)
retriever = vectorstore.as_retriever()

template = """Responda à pergunta com base apenas no seguinte contexto:
{contexto}

Pergunta: {pergunta}
"""
prompt = ChatPromptTemplate.from_template(template)
# model = ChatOpenAI()
model = ChatGroq(model_name="mixtral-8x7b-32768")
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"contexto": retriever, "pergunta": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("Onde o Marcos trabalhou?")